# E03_BS Experiments - Batch Size Analysis

This notebook analyzes and compares the results from the E03_BS experiment variants with different batch sizes.

Expected variants:
- Different batch sizes (32, 64, 512, 1024, 2048)
- Analysis of how batch size affects training dynamics, win rates, and convergence

In [137]:
import sys
from pathlib import Path

sys.path.append(str(Path("../..").resolve()))
import os

if os.path.basename(os.getcwd()) == "analysis":
    os.chdir("../..")
elif os.path.basename(os.getcwd()) == "E03_BS":
    os.chdir("../..")

print(f"Current directory: {os.getcwd()}")

Current directory: c:\Users\Laboratorio IA\OneDrive - Escuela Politécnica Nacional\doctorado\hierarchical-SAE


In [138]:
import pickle
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from glob import glob

# Find all E03 BATCH_SIZE pickle files
pickle_files = glob("analysis\E03_BS\E03_BATCH_SIZE*.pkl")
print(f"Found {len(pickle_files)} pickle files:")
for f in sorted(pickle_files):
    print(f"  - {f}")

Found 4 pickle files:
  - analysis\E03_BS\E03_BATCH_SIZE1024.pkl
  - analysis\E03_BS\E03_BATCH_SIZE32.pkl
  - analysis\E03_BS\E03_BATCH_SIZE512.pkl
  - analysis\E03_BS\E03_BATCH_SIZE64.pkl


## Load Data from Pickle Files

In [139]:
# Load all experiment data
evals = {}

for file in sorted(pickle_files):
    # Extract batch size from filename (e.g., "32" from "E03_BATCH_SIZE32.pkl")
    import re

    match = re.search(r"E03_BATCH_SIZE(\d+)\.pkl", file)
    if match:
        batch_size = match.group(1)
    else:
        batch_size = "unknown"

    try:
        with open(file, "rb") as f:
            evals[batch_size] = pickle.load(f)
        print(f"Loaded {file} as variant 'BS_{batch_size}'")
    except Exception as e:
        print(f"Error loading {file}: {e}")

# Display structure of first variant
if evals:
    first_variant = list(evals.keys())[0]
    print(f"\nData structure for variant BS_{first_variant}:")
    print(evals[first_variant].keys())

    # Display available batch sizes
    print(f"\nAvailable batch sizes: {sorted(evals.keys(), key=lambda x: int(x))}")

Loaded analysis\E03_BS\E03_BATCH_SIZE1024.pkl as variant 'BS_1024'
Loaded analysis\E03_BS\E03_BATCH_SIZE32.pkl as variant 'BS_32'
Loaded analysis\E03_BS\E03_BATCH_SIZE512.pkl as variant 'BS_512'
Loaded analysis\E03_BS\E03_BATCH_SIZE64.pkl as variant 'BS_64'

Data structure for variant BS_1024:
dict_keys(['epochs_results', 'loss_values', 'win_rate'])

Available batch sizes: ['32', '64', '512', '1024']


## Win Rate Comparison Across Batch Sizes

In [140]:
# Define consistent colors for each bot type
bot_colors = {
    "bot_good": "#1f77b4",  # Blue
    "bot_good_WR_B": "#1f77b4",  # Blue
    "bot_medium": "#ff7f0e",  # Orange
    "bot_random": "#2ca02c",  # Green
    "bot_bad": "#d62728",  # Red
    "bot_Michael": "#9467bd",  # Purple
    "bot_Michael2": "#8c564b",  # Brown
}

fig = go.Figure()

for batch_size, data in sorted(evals.items(), key=lambda x: int(x[0])):
    if "win_rate" not in data:
        continue

    win_rate = data["win_rate"]

    for bot, rates in win_rate.items():
        bot_color = bot_colors.get(bot, "#999999")
        fig.add_trace(
            go.Scatter(
                y=rates,
                mode="lines",
                name=f"BS_{batch_size} - {bot}",
                line=dict(color=bot_color),
                opacity=0.7,
            )
        )

fig.update_layout(
    title="Win Rate per Epoch - All Batch Size Variants",
    xaxis_title="Epoch",
    yaxis_title="Win Rate",
    legend_title="Variant - Bot",
    height=600,
)
fig.show()

## Loss Values Comparison

In [141]:
fig = make_subplots(
    rows=2,
    cols=1,
    subplot_titles=("Loss Values", "Loss Values (Log Scale)"),
    vertical_spacing=0.15,
)

for batch_size, data in sorted(evals.items(), key=lambda x: int(x[0])):
    if "loss_values" not in data:
        continue

    loss_data = data["loss_values"]

    # Extract loss at epoch boundaries
    if "epoch_values" in loss_data and "loss_values" in loss_data:
        loss_epoch = [loss_data["loss_values"][j] for j in loss_data["epoch_values"]]

        # Regular scale
        fig.add_trace(
            go.Scatter(
                y=loss_epoch,
                mode="markers",
                name=f"BS_{batch_size}",
                marker=dict(opacity=0.35, size=4),
            ),
            row=1,
            col=1,
        )

        # Log scale
        fig.add_trace(
            go.Scatter(
                y=loss_epoch,
                mode="lines",
                name=f"BS_{batch_size}",
                showlegend=False,
            ),
            row=2,
            col=1,
        )

fig.update_layout(height=800, title_text="Loss Comparison Across Batch Sizes")
fig.update_yaxes(title_text="Loss", row=1, col=1)
fig.update_yaxes(title_text="Loss (log)", type="log", row=2, col=1)
fig.update_xaxes(title_text="Epoch", row=2, col=1)
fig.show()

## Detailed Analysis per Variant (Win Rate + Loss)

In [142]:
# Smooth win rates function
def smooth_win_rates(rates, window_size=11):
    smoothed = []
    for i in range(len(rates)):
        window = rates[
            max(0, i - window_size // 2) : min(len(rates), i + window_size // 2 + 1)
        ]
        smoothed.append(sum(window) / len(window))
    return smoothed


# Create one subplot per variant
fig = make_subplots(
    rows=len(evals),
    cols=1,
    subplot_titles=[
        f"Batch Size {bs}" for bs in sorted(evals.keys(), key=lambda x: int(x))
    ],
    vertical_spacing=0.08,
)

for i, (batch_size, data) in enumerate(
    sorted(evals.items(), key=lambda x: int(x[0])), 1
):
    if "win_rate" not in data:
        continue

    win_rate = data["win_rate"]

    # Plot each bot's win rate
    for bot, rates in win_rate.items():
        bot_color = bot_colors.get(bot, "#999999")

        # Raw data as markers
        fig.add_trace(
            go.Scatter(
                y=rates,
                mode="markers",
                name=f"{bot} - BS_{batch_size}",
                marker=dict(size=3, color=bot_color, opacity=0.5),
                legendgroup=f"group{batch_size}",
                showlegend=True,
            ),
            row=i,
            col=1,
        )

        # Smoothed line
        smoothed = smooth_win_rates(rates)
        fig.add_trace(
            go.Scatter(
                y=smoothed,
                mode="lines",
                name=f"{bot} - BS_{batch_size} (smooth)",
                line=dict(width=3, color=bot_color),
                showlegend=False,
            ),
            row=i,
            col=1,
        )

    # Add loss if available
    if "loss_values" in data:
        loss_data = data["loss_values"]
        if "epoch_values" in loss_data and "loss_values" in loss_data:
            loss_epoch = [
                loss_data["loss_values"][j] for j in loss_data["epoch_values"]
            ]
            fig.add_trace(
                go.Scatter(
                    y=loss_epoch,
                    mode="lines",
                    name=f"BS_{batch_size} - Loss",
                    line=dict(dash="dash"),
                    legendgroup=f"group{batch_size}",
                    showlegend=True,
                ),
                row=i,
                col=1,
            )

    # Update y-axis
    fig.update_yaxes(title_text="Win Rate / Loss", row=i, col=1)

# Update x-axis (only for bottom plot)
fig.update_xaxes(title_text="Epoch", row=len(evals), col=1)

fig.update_layout(
    height=400 * len(evals),
    title_text="Detailed Win Rate and Loss Analysis per Batch Size Variant",
)
fig.show()

## Win Rate Comparison by Opponent Bot

In [143]:
# Get all unique bots from all variants
all_bots = set()
for data in evals.values():
    if "win_rate" in data:
        all_bots.update(data["win_rate"].keys())
all_bots = sorted(list(all_bots))

# Create subplots - one per bot type
fig = make_subplots(
    rows=len(all_bots),
    cols=1,
    subplot_titles=[f"Win Rate vs {bot}" for bot in all_bots],
    vertical_spacing=0.1,
)

# Define colors for each batch size variant
variant_colors = {
    "32": "#1f77b4",  # Blue
    "64": "#ff7f0e",  # Orange
    "512": "#2ca02c",  # Green
    "1024": "#d62728",  # Red
    "2048": "#9467bd",  # Purple
}

for bot_idx, bot in enumerate(all_bots, 1):
    for batch_size, data in sorted(evals.items(), key=lambda x: int(x[0])):
        if "win_rate" not in data:
            continue

        win_rate = data["win_rate"]

        if bot in win_rate:
            rates = win_rate[bot]
            variant_color = variant_colors.get(batch_size, "#999999")

            # Raw data as markers
            fig.add_trace(
                go.Scatter(
                    y=rates,
                    mode="markers",
                    name=f"BS_{batch_size}",
                    marker=dict(size=3, color=variant_color, opacity=0.3),
                    legendgroup=f"group{batch_size}",
                    showlegend=(bot_idx == 1),
                ),
                row=bot_idx,
                col=1,
            )

            # Smoothed line
            smoothed = smooth_win_rates(rates)
            fig.add_trace(
                go.Scatter(
                    y=smoothed,
                    mode="lines",
                    name=f"BS_{batch_size} smooth",
                    line=dict(width=2, color=variant_color),
                    opacity=0.7,
                    legendgroup=f"group{batch_size}",
                    showlegend=False,
                ),
                row=bot_idx,
                col=1,
            )

    # Update y-axis
    fig.update_yaxes(title_text="Win Rate", row=bot_idx, col=1)

# Update x-axis (only for bottom plot)
fig.update_xaxes(title_text="Epoch", row=len(all_bots), col=1)

fig.update_layout(
    height=300 * len(all_bots),
    title_text="Win Rate Comparison by Opponent Bot - All Batch Size Variants",
    xaxis=dict(range=[0, 100]),
)
fig.show()

## Statistical Summary

In [144]:
stats = {}

for batch_size, data in sorted(evals.items(), key=lambda x: int(x[0])):
    if "win_rate" not in data:
        continue

    win_rate = data["win_rate"]
    stats[f"BS_{batch_size}"] = {}

    for bot, rates in win_rate.items():
        rates_array = np.array(rates)
        stats[f"BS_{batch_size}"][bot] = {
            "mean_win_rate": np.mean(rates_array),
            "final_win_rate": np.mean(rates_array[-20:]),  # Last 20 epochs
            "std_win_rate": np.std(rates_array),
            "max_win_rate": np.max(rates_array),
            "min_win_rate": np.min(rates_array),
            "total_epochs": len(rates_array),
        }

# Convert to DataFrame
if stats:
    stats_df = pd.DataFrame.from_dict(
        {
            (variant, bot): stats[variant][bot]
            for variant in stats.keys()
            for bot in stats[variant].keys()
        },
        orient="index",
    )
    stats_df.index.names = ["Variant", "Bot"]
    stats_df = stats_df.round(4)
    display(stats_df)

mean_win_rate  final_win_rate  std_win_rate  \
Variant Bot                                                          
BS_32   bot_good_WR_B         0.3688          0.3933        0.0879   
        bot_random            0.4957          0.5033        0.0912   
BS_64   bot_good_WR_B         0.3674          0.3758        0.0857   
        bot_random            0.4943          0.5008        0.0906   
BS_512  bot_good_WR_B         0.3718          0.3925        0.0924   
        bot_random            0.4996          0.5267        0.0922   
BS_1024 bot_good_WR_B         0.3748          0.3442        0.0900   
        bot_random            0.5040          0.4633        0.0938   

                       max_win_rate  min_win_rate  total_epochs  
Variant Bot                                                      
BS_32   bot_good_WR_B        0.6333        0.1333          1000  
        bot_random           0.8000        0.2333          1000  
BS_64   bot_good_WR_B        0.6667        0.1333          1000  
        bot_random           0.7667        0.2000          1000  
BS_512  bot_good_WR_B        0.6667        0.0667          1000  
        bot_random           0.7333        0.2000          1000  
BS_1024 bot_good_WR_B        0.7667        0.1333          1000  
        bot_random           0.7667        0.2000          1000

## Loss Statistics Summary

In [145]:
loss_stats = {}

for batch_size, data in sorted(evals.items(), key=lambda x: int(x[0])):
    if "loss_values" not in data:
        continue

    loss_data = data["loss_values"]

    if "epoch_values" in loss_data and "loss_values" in loss_data:
        loss_epoch = np.array(
            [loss_data["loss_values"][j] for j in loss_data["epoch_values"]]
        )

        loss_stats[f"BS_{batch_size}"] = {
            "mean_loss": np.mean(loss_epoch),
            "final_loss": np.mean(loss_epoch[-20:]),
            "std_loss": np.std(loss_epoch),
            "min_loss": np.min(loss_epoch),
            "max_loss": np.max(loss_epoch),
        }

if loss_stats:
    loss_stats_df = pd.DataFrame.from_dict(loss_stats, orient="index")
    loss_stats_df.index.name = "Variant"
    loss_stats_df = loss_stats_df.round(6)
    display(loss_stats_df)

,mean_loss,final_loss,std_loss,min_loss,max_loss
Variant,,,,,
BS_32,0.400192,0.453752,0.109948,0.004294,0.573576
BS_64,0.398421,0.456153,0.104261,0.003527,0.531693
BS_512,0.398914,0.461719,0.097555,0.019203,0.486682
BS_1024,0.400120,0.459872,0.094836,0.025511,0.479842


## Correlation Analysis Between Variants

In [146]:
# Prepare win rate data for correlation analysis
if len(evals) > 1 and all("win_rate" in data for data in evals.values()):
    # Get common bots across all variants
    common_bots = set.intersection(
        *[set(data["win_rate"].keys()) for data in evals.values()]
    )

    correlation_results = {}

    for bot in common_bots:
        # Collect win rates for each variant for the current bot
        win_rate_matrix = pd.DataFrame(
            {
                variant: evals[variant]["win_rate"][bot]
                for variant in evals.keys()
                if bot in evals[variant]["win_rate"]
            }
        )
        # Compute correlation matrix
        correlation_results[bot] = win_rate_matrix.corr()

    # Display correlation matrices for each rival bot
    for bot, corr_matrix in correlation_results.items():
        print(f"\nCorrelation matrix for {bot}:")
        display(corr_matrix.round(4))


Correlation matrix for bot_good_WR_B:


,1024,32,512,64
1024,1.0000,-0.0314,0.0145,0.0452
32,-0.0314,1.0000,-0.0288,-0.0017
512,0.0145,-0.0288,1.0000,-0.0057
64,0.0452,-0.0017,-0.0057,1.0000



Correlation matrix for bot_random:


,1024,32,512,64
1024,1.0000,0.0098,0.0110,0.0209
32,0.0098,1.0000,-0.0893,0.0524
512,0.0110,-0.0893,1.0000,0.0454
64,0.0209,0.0524,0.0454,1.0000


## Training Efficiency Analysis

In [147]:
# Compare training efficiency across variants
efficiency_data = {}

for batch_size, data in sorted(evals.items(), key=lambda x: int(x[0])):
    efficiency_data[f"BS_{batch_size}"] = {}

    # Win rate metrics
    if "win_rate" in data:
        all_rates = []
        for bot, rates in data["win_rate"].items():
            all_rates.extend(rates)
        efficiency_data[f"BS_{batch_size}"]["avg_win_rate"] = np.mean(all_rates)
        efficiency_data[f"BS_{batch_size}"]["num_bots"] = len(data["win_rate"])

    # Loss metrics
    if "loss_values" in data:
        loss_data = data["loss_values"]
        if "loss_values" in loss_data:
            efficiency_data[f"BS_{batch_size}"]["final_loss"] = (
                loss_data["loss_values"][-1] if loss_data["loss_values"] else None
            )

    # Epoch count
    if "epochs_results" in data:
        efficiency_data[f"BS_{batch_size}"]["total_epochs"] = len(
            data["epochs_results"]
        )

    # Add batch size as numeric value
    efficiency_data[f"BS_{batch_size}"]["batch_size"] = int(batch_size)

if efficiency_data:
    efficiency_df = pd.DataFrame.from_dict(efficiency_data, orient="index")
    efficiency_df.index.name = "Variant"
    efficiency_df = efficiency_df.sort_values("batch_size")
    display(efficiency_df.round(4))

,avg_win_rate,num_bots,final_loss,total_epochs,batch_size
Variant,,,,,
BS_32,0.4322,2,0.5001,1000,32
BS_64,0.4309,2,0.4376,1000,64
BS_512,0.4357,2,0.4570,1000,512
BS_1024,0.4394,2,0.4545,1000,1024


## Summary and Conclusions

Key findings from the E03_BS batch size analysis:

1. **Win Rates**: Compare performance across different batch sizes (32, 64, 512, 1024, 2048)
2. **Training Stability**: Evaluate variance in win rates for each batch size
3. **Loss Convergence**: Analyze how batch size affects loss reduction
4. **Computational Efficiency**: Consider training time vs. performance trade-offs

The analysis above provides detailed metrics for each batch size variant.

## Swiss Tournament Results

Analysis of Swiss tournament rankings for different batch size variants.

In [148]:
from glob import glob
import re

# Find all Swiss tournament result files
swiss_files = glob("analysis\E03_BS\swiss_tournament_*.pkl")
print(f"Found {len(swiss_files)} Swiss tournament files:")
for f in sorted(swiss_files):
    print(f"  - {f}")

# Load Swiss tournament data
swiss_tournaments = {}

for file in sorted(swiss_files):
    # Extract timestamp or identifier from filename
    match = re.search(r"swiss_tournament_(\d+_\d+)\.pkl", file)
    if match:
        tournament_id = match.group(1)
    else:
        tournament_id = file.replace("swiss_tournament_", "").replace(".pkl", "")

    try:
        with open(file, "rb") as f:
            swiss_tournaments[tournament_id] = pickle.load(f)
        print(f"Loaded {file} as tournament '{tournament_id}'")
    except Exception as e:
        print(f"Error loading {file}: {e}")

if swiss_tournaments:
    print(f"\nLoaded {len(swiss_tournaments)} Swiss tournaments")
    first_tournament = list(swiss_tournaments.keys())[0]
    print(f"\nData structure for tournament {first_tournament}:")
    print(swiss_tournaments[first_tournament].keys())

Found 4 Swiss tournament files:
  - analysis\E03_BS\swiss_tournament_20251031_164218.pkl
  - analysis\E03_BS\swiss_tournament_20251031_164219.pkl
  - analysis\E03_BS\swiss_tournament_20251031_164226.pkl
  - analysis\E03_BS\swiss_tournament_20251031_164228.pkl
Loaded analysis\E03_BS\swiss_tournament_20251031_164218.pkl as tournament '20251031_164218'
Loaded analysis\E03_BS\swiss_tournament_20251031_164219.pkl as tournament '20251031_164219'
Loaded analysis\E03_BS\swiss_tournament_20251031_164226.pkl as tournament '20251031_164226'
Loaded analysis\E03_BS\swiss_tournament_20251031_164228.pkl as tournament '20251031_164228'

Loaded 4 Swiss tournaments

Data structure for tournament 20251031_164218:
dict_keys(['config', 'final_standings', 'match_results', 'timestamp'])


### Swiss Tournament Configuration

In [149]:
# Display tournament configurations
if swiss_tournaments:
    config_data = {}

    for tournament_id, data in swiss_tournaments.items():
        if "config" in data:
            config = data["config"]
            config_data[tournament_id] = {
                "num_rounds": config.get("num_rounds", "N/A"),
                "double_swiss": config.get("double_swiss", "N/A"),
                "mcmahon": config.get("mcmahon", "N/A"),
                "folder": config.get("folder", "N/A"),
                "deterministic": config.get("bot_params", {}).get(
                    "deterministic", "N/A"
                ),
                "temperature": config.get("bot_params", {}).get("temperature", "N/A"),
            }

    config_df = pd.DataFrame.from_dict(config_data, orient="index")
    config_df.index.name = "Tournament ID"
    print("\nTournament Configurations:")
    display(config_df)


Tournament Configurations:


,num_rounds,double_swiss,mcmahon,folder,deterministic,temperature
Tournament ID,,,,,,
20251031_164218,50,True,True,CHECKPOINTS//E03_BATCH_SIZE512,False,0.1
20251031_164219,50,True,True,CHECKPOINTS//E03_BATCH_SIZE1024,False,0.1
20251031_164226,50,True,True,CHECKPOINTS//E03_BATCH_SIZE64,False,0.1
20251031_164228,50,True,True,CHECKPOINTS//E03_BATCH_SIZE32,False,0.1


### Final Rankings Analysis

In [150]:
# Extract and analyze final standings from Swiss tournaments
if swiss_tournaments:
    for tournament_id, data in swiss_tournaments.items():
        if "final_standings" not in data:
            continue

        standings = data["final_standings"]

        # Extract batch size from checkpoint names
        standings_with_bs = []
        for bot in standings[:20]:  # Top 20
            # Try to extract batch size from checkpoint name
            bs_match = re.search(r"BATCH_SIZE(\d+)", bot["name"])
            if bs_match:
                batch_size = int(bs_match.group(1))
            else:
                batch_size = None

            standings_with_bs.append(
                {
                    "name": bot["name"][-40:],  # Truncate for display
                    "batch_size": batch_size,
                    "epoch": bot.get("epoch", 0),
                    "score": bot.get("score", 0),
                    "wins": bot.get("wins", 0),
                    "draws": bot.get("draws", 0),
                    "losses": bot.get("losses", 0),
                    "matches": bot.get("matches_played", 0),
                }
            )

        print(f"\n{'='*80}")
        print(f"Tournament {tournament_id} - Top 20 Rankings")
        print(f"{'='*80}")

        standings_df = pd.DataFrame(standings_with_bs)
        standings_df.index = range(1, len(standings_df) + 1)
        standings_df.index.name = "Rank"
        display(standings_df)


Tournament 20251031_164218 - Top 20 Rankings


,name,batch_size,epoch,score,wins,draws,losses,matches
Rank,,,,,,,,
1,20251030_1735-E03_BATCH_SIZE512_E_0001,512,0,69.0,69,0,31,100
2,20251030_1736-E03_BATCH_SIZE512_E_0013,512,0,66.0,66,0,34,100
3,20251030_1735-E03_BATCH_SIZE512_E_0002,512,0,66.0,65,2,33,100
4,20251030_1820-E03_BATCH_SIZE512_E_0399,512,0,63.5,63,1,36,100
5,20251030_1735-E03_BATCH_SIZE512_E_0000,512,0,62.0,62,0,38,100
6,20251030_1829-E03_BATCH_SIZE512_E_0476,512,0,62.0,62,0,38,100
7,20251030_1915-E03_BATCH_SIZE512_E_0869,512,0,62.0,62,0,38,100
8,20251030_1745-E03_BATCH_SIZE512_E_0091,512,0,62.0,62,0,38,100
9,20251030_1812-E03_BATCH_SIZE512_E_0331,512,0,62.0,62,0,38,100



Tournament 20251031_164219 - Top 20 Rankings


,name,batch_size,epoch,score,wins,draws,losses,matches
Rank,,,,,,,,
1,20251030_1736-E03_BATCH_SIZE1024_E_0009,1024,0,65.0,65,0,35,100
2,20251030_1757-E03_BATCH_SIZE1024_E_0200,1024,0,65.0,65,0,35,100
3,20251030_1736-E03_BATCH_SIZE1024_E_0015,1024,0,64.0,64,0,36,100
4,20251030_1735-E03_BATCH_SIZE1024_E_0006,1024,0,64.0,64,0,36,100
5,20251030_1735-E03_BATCH_SIZE1024_E_0001,1024,0,63.0,63,0,37,100
6,20251030_1854-E03_BATCH_SIZE1024_E_0688,1024,0,63.0,63,0,37,100
7,20251030_1737-E03_BATCH_SIZE1024_E_0025,1024,0,63.0,63,0,37,100
8,20251030_1737-E03_BATCH_SIZE1024_E_0017,1024,0,63.0,62,2,36,100
9,20251030_1846-E03_BATCH_SIZE1024_E_0625,1024,0,62.5,62,1,37,100



Tournament 20251031_164226 - Top 20 Rankings


,name,batch_size,epoch,score,wins,draws,losses,matches
Rank,,,,,,,,
1,20251030_1812-E03_BATCH_SIZE64_E_0312,64,0,65.0,65,0,35,100
2,20251030_1755-E03_BATCH_SIZE64_E_0167,64,0,64.5,64,1,35,100
3,20251030_1907-E03_BATCH_SIZE64_E_0751,64,0,64.0,64,0,36,100
4,20251030_1813-E03_BATCH_SIZE64_E_0315,64,0,64.0,63,2,35,100
5,20251030_1841-E03_BATCH_SIZE64_E_0548,64,0,63.0,63,0,37,100
6,20251030_1918-E03_BATCH_SIZE64_E_0841,64,0,63.0,63,0,37,100
7,20251030_1806-E03_BATCH_SIZE64_E_0258,64,0,62.5,62,1,37,100
8,20251030_1751-E03_BATCH_SIZE64_E_0134,64,0,62.0,62,0,38,100
9,20251030_1751-E03_BATCH_SIZE64_E_0139,64,0,62.0,62,0,38,100



Tournament 20251031_164228 - Top 20 Rankings


,name,batch_size,epoch,score,wins,draws,losses,matches
Rank,,,,,,,,
1,20251030_1759-E03_BATCH_SIZE32_E_0193,32,0,65.5,65,1,34,100
2,20251030_1747-E03_BATCH_SIZE32_E_0101,32,0,65.0,65,0,35,100
3,20251030_1814-E03_BATCH_SIZE32_E_0308,32,0,65.0,65,0,35,100
4,20251030_1932-E03_BATCH_SIZE32_E_0882,32,0,63.0,63,0,37,100
5,20251030_1739-E03_BATCH_SIZE32_E_0037,32,0,62.5,62,1,37,100
6,20251030_1824-E03_BATCH_SIZE32_E_0385,32,0,62.0,62,0,38,100
7,20251030_1937-E03_BATCH_SIZE32_E_0921,32,0,62.0,62,0,38,100
8,20251030_1855-E03_BATCH_SIZE32_E_0612,32,0,62.0,62,0,38,100
9,20251030_1742-E03_BATCH_SIZE32_E_0062,32,0,62.0,62,0,38,100


### Batch Size Performance in Tournaments

In [151]:
# Analyze how different batch sizes performed in tournaments
if swiss_tournaments:
    for tournament_id, data in swiss_tournaments.items():
        if "final_standings" not in data:
            continue

        standings = data["final_standings"]

        # Group by batch size
        bs_performance = {}

        for bot in standings:
            bs_match = re.search(r"BATCH_SIZE(\d+)", bot["name"])
            if bs_match:
                batch_size = bs_match.group(1)

                if batch_size not in bs_performance:
                    bs_performance[batch_size] = {
                        "count": 0,
                        "total_score": 0,
                        "avg_score": 0,
                        "best_rank": len(standings) + 1,
                        "avg_rank": 0,
                        "total_wins": 0,
                        "total_draws": 0,
                        "total_losses": 0,
                    }

                rank = standings.index(bot) + 1
                bs_performance[batch_size]["count"] += 1
                bs_performance[batch_size]["total_score"] += bot.get("score", 0)
                bs_performance[batch_size]["best_rank"] = min(
                    bs_performance[batch_size]["best_rank"], rank
                )
                bs_performance[batch_size]["avg_rank"] += rank
                bs_performance[batch_size]["total_wins"] += bot.get("wins", 0)
                bs_performance[batch_size]["total_draws"] += bot.get("draws", 0)
                bs_performance[batch_size]["total_losses"] += bot.get("losses", 0)

        # Calculate averages
        for bs in bs_performance:
            count = bs_performance[bs]["count"]
            if count > 0:
                bs_performance[bs]["avg_score"] = (
                    bs_performance[bs]["total_score"] / count
                )
                bs_performance[bs]["avg_rank"] = bs_performance[bs]["avg_rank"] / count

        print(f"\n{'='*80}")
        print(f"Tournament {tournament_id} - Performance by Batch Size")
        print(f"{'='*80}")

        bs_df = pd.DataFrame.from_dict(bs_performance, orient="index")
        bs_df.index.name = "Batch Size"
        bs_df = bs_df.sort_index()
        bs_df = bs_df.round(2)
        display(bs_df)


Tournament 20251031_164218 - Performance by Batch Size


,count,total_score,avg_score,best_rank,avg_rank,total_wins,total_draws,total_losses
Batch Size,,,,,,,,
512,1001,50050.0,50.0,1,501.0,49864,272,49864



Tournament 20251031_164219 - Performance by Batch Size


,count,total_score,avg_score,best_rank,avg_rank,total_wins,total_draws,total_losses
Batch Size,,,,,,,,
1024,1001,50050.0,50.0,1,501.0,49850,300,49850



Tournament 20251031_164226 - Performance by Batch Size


,count,total_score,avg_score,best_rank,avg_rank,total_wins,total_draws,total_losses
Batch Size,,,,,,,,
64,1001,50050.0,50.0,1,501.0,49843,314,49843



Tournament 20251031_164228 - Performance by Batch Size


,count,total_score,avg_score,best_rank,avg_rank,total_wins,total_draws,total_losses
Batch Size,,,,,,,,
32,1001,50050.0,50.0,1,501.0,49859,282,49859


### Top Performers by Batch Size

In [152]:
# Find best checkpoint for each batch size in each tournament
if swiss_tournaments:
    for tournament_id, data in swiss_tournaments.items():
        if "final_standings" not in data:
            continue

        standings = data["final_standings"]

        # Find best performer for each batch size
        best_by_bs = {}

        for rank, bot in enumerate(standings, 1):
            bs_match = re.search(r"BATCH_SIZE(\d+)", bot["name"])
            if bs_match:
                batch_size = bs_match.group(1)

                if batch_size not in best_by_bs:
                    best_by_bs[batch_size] = {
                        "rank": rank,
                        "name": bot["name"][-50:],
                        "epoch": bot.get("epoch", 0),
                        "score": bot.get("score", 0),
                        "wins": bot.get("wins", 0),
                        "draws": bot.get("draws", 0),
                        "losses": bot.get("losses", 0),
                    }

        print(f"\n{'='*80}")
        print(f"Tournament {tournament_id} - Best Checkpoint per Batch Size")
        print(f"{'='*80}")

        best_df = pd.DataFrame.from_dict(best_by_bs, orient="index")
        best_df.index.name = "Batch Size"
        best_df = best_df.sort_values("rank")
        display(best_df)


Tournament 20251031_164218 - Best Checkpoint per Batch Size


,rank,name,epoch,score,wins,draws,losses
Batch Size,,,,,,,
512,1,20251030_1735-E03_BATCH_SIZE512_E_0001,0,69,69,0,31



Tournament 20251031_164219 - Best Checkpoint per Batch Size


,rank,name,epoch,score,wins,draws,losses
Batch Size,,,,,,,
1024,1,20251030_1736-E03_BATCH_SIZE1024_E_0009,0,65,65,0,35



Tournament 20251031_164226 - Best Checkpoint per Batch Size


,rank,name,epoch,score,wins,draws,losses
Batch Size,,,,,,,
64,1,20251030_1812-E03_BATCH_SIZE64_E_0312,0,65,65,0,35



Tournament 20251031_164228 - Best Checkpoint per Batch Size


,rank,name,epoch,score,wins,draws,losses
Batch Size,,,,,,,
32,1,20251030_1759-E03_BATCH_SIZE32_E_0193,0,65.5,65,1,34


### Visualize Tournament Rankings Distribution

In [153]:
# Visualize rank distribution by batch size
if swiss_tournaments:
    for tournament_id, data in swiss_tournaments.items():
        if "final_standings" not in data:
            continue

        standings = data["final_standings"]

        # Collect ranks by batch size
        ranks_by_bs = {}

        for rank, bot in enumerate(standings, 1):
            bs_match = re.search(r"BATCH_SIZE(\d+)", bot["name"])
            if bs_match:
                batch_size = bs_match.group(1)

                if batch_size not in ranks_by_bs:
                    ranks_by_bs[batch_size] = []

                ranks_by_bs[batch_size].append(rank)

        # Create box plot
        fig = go.Figure()

        for bs in sorted(ranks_by_bs.keys(), key=lambda x: int(x)):
            fig.add_trace(go.Box(y=ranks_by_bs[bs], name=f"BS_{bs}", boxmean="sd"))

        fig.update_layout(
            title=f"Tournament {tournament_id} - Rank Distribution by Batch Size",
            yaxis_title="Rank (lower is better)",
            xaxis_title="Batch Size",
            height=500,
            yaxis=dict(autorange="reversed"),  # Lower ranks at top
        )
        fig.show()

### Score Distribution Visualization

In [154]:
# Visualize score distribution by batch size
if swiss_tournaments:
    for tournament_id, data in swiss_tournaments.items():
        if "final_standings" not in data:
            continue

        standings = data["final_standings"]

        # Collect scores by batch size
        scores_by_bs = {}

        for bot in standings:
            bs_match = re.search(r"BATCH_SIZE(\d+)", bot["name"])
            if bs_match:
                batch_size = bs_match.group(1)

                if batch_size not in scores_by_bs:
                    scores_by_bs[batch_size] = []

                scores_by_bs[batch_size].append(bot.get("score", 0))

        # Create violin plot
        fig = go.Figure()

        for bs in sorted(scores_by_bs.keys(), key=lambda x: int(x)):
            fig.add_trace(
                go.Violin(
                    y=scores_by_bs[bs],
                    name=f"BS_{bs}",
                    box_visible=True,
                    meanline_visible=True,
                )
            )

        fig.update_layout(
            title=f"Tournament {tournament_id} - Score Distribution by Batch Size",
            yaxis_title="Score",
            xaxis_title="Batch Size",
            height=500,
        )
        fig.show()

### Match Results Analysis

In [155]:
# Analyze head-to-head performance between different batch sizes
if swiss_tournaments:
    for tournament_id, data in swiss_tournaments.items():
        if "match_results" not in data:
            continue

        matches = data["match_results"]

        # Create a matrix of wins between batch sizes
        h2h_matrix = {}

        for match in matches:
            # Extract batch sizes from player names
            p1_match = re.search(r"BATCH_SIZE(\d+)", match.get("player1", ""))
            p2_match = re.search(r"BATCH_SIZE(\d+)", match.get("player2", ""))

            if p1_match and p2_match:
                bs1 = p1_match.group(1)
                bs2 = p2_match.group(1)

                if bs1 not in h2h_matrix:
                    h2h_matrix[bs1] = {}
                if bs2 not in h2h_matrix[bs1]:
                    h2h_matrix[bs1][bs2] = {"wins": 0, "losses": 0, "draws": 0}

                p1_wins = match.get("p1_wins", 0)
                p2_wins = match.get("p2_wins", 0)
                draws = match.get("draws", 0)

                if p1_wins > p2_wins:
                    h2h_matrix[bs1][bs2]["wins"] += 1
                elif p2_wins > p1_wins:
                    h2h_matrix[bs1][bs2]["losses"] += 1
                else:
                    h2h_matrix[bs1][bs2]["draws"] += 1

        # Create win rate matrix
        if h2h_matrix:
            all_bs = sorted(set(h2h_matrix.keys()), key=lambda x: int(x))

            win_rate_matrix = np.zeros((len(all_bs), len(all_bs)))

            for i, bs1 in enumerate(all_bs):
                for j, bs2 in enumerate(all_bs):
                    if bs1 in h2h_matrix and bs2 in h2h_matrix[bs1]:
                        stats = h2h_matrix[bs1][bs2]
                        total = stats["wins"] + stats["losses"] + stats["draws"]
                        if total > 0:
                            win_rate_matrix[i, j] = (
                                stats["wins"] + 0.5 * stats["draws"]
                            ) / total

            # Create heatmap
            fig = go.Figure(
                data=go.Heatmap(
                    z=win_rate_matrix,
                    x=[f"BS_{bs}" for bs in all_bs],
                    y=[f"BS_{bs}" for bs in all_bs],
                    colorscale="RdYlGn",
                    text=np.round(win_rate_matrix, 3),
                    texttemplate="%{text}",
                    textfont={"size": 10},
                    colorbar=dict(title="Win Rate"),
                )
            )

            fig.update_layout(
                title=f"Tournament {tournament_id} - Head-to-Head Win Rate Matrix",
                xaxis_title="Opponent Batch Size",
                yaxis_title="Player Batch Size",
                height=600,
                width=700,
            )
            fig.show()

## Swiss Tournament Summary

Key findings from Swiss tournament analysis:

1. **Best Batch Size**: Identify which batch size produced the strongest checkpoints
2. **Consistency**: Evaluate which batch size variants performed consistently across epochs
3. **Head-to-Head**: Analyze matchups between different batch sizes
4. **Optimal Epochs**: Determine at which epoch each batch size peaked

The tournament results provide a comprehensive ranking based on actual gameplay performance.

In [158]:
# Print relative paths of the top 10 checkpoints for each batch size variant from ALL tournaments

if swiss_tournaments:
    print(f"Found {len(swiss_tournaments)} tournament(s)\n")

    # Process each tournament
    for tournament_id in sorted(swiss_tournaments.keys()):
        standings_src = swiss_tournaments[tournament_id].get("final_standings", [])

        print("=" * 80)
        print(f"TOURNAMENT: {tournament_id}")
        print(f"Total checkpoints: {len(standings_src)}")
        print("=" * 80)

        # Group top 10 paths by batch size for this tournament
        top_paths_by_bs = {}

        # Iterate through ALL standings entries
        for rank, entry in enumerate(standings_src, 1):
            name = entry.get("name", "")
            path = entry.get("path", name)

            # Extract batch size from name or path
            m = re.search(r"BATCH_SIZE(\d+)", name) or re.search(
                r"BATCH_SIZE(\d+)", path
            )

            if not m:
                continue

            bs = m.group(1)

            # Initialize list for this batch size if it doesn't exist
            if bs not in top_paths_by_bs:
                top_paths_by_bs[bs] = []

            # Only add if we haven't reached 10 for this batch size yet
            if len(top_paths_by_bs[bs]) < 10:
                top_paths_by_bs[bs].append(
                    {
                        "rank": rank,
                        "path": path,
                        "epoch": entry.get("epoch", "N/A"),
                        "score": entry.get("score", 0),
                    }
                )

        # Print results for this tournament
        print("\nTOP 10 CHECKPOINTS PER BATCH SIZE")
        print("-" * 80)

        for bs in sorted(top_paths_by_bs.keys(), key=lambda x: int(x)):
            print(f"\nBatch Size {bs}:")
            for idx, info in enumerate(top_paths_by_bs[bs], 1):
                print(
                    f"  {idx}. Rank {info['rank']:3d} | Epoch {info['epoch']:4} | Score {info['score']:5.1f} | {info['path']}"
                )

        # Debug info for this tournament
        print(
            f"\n[Debug] Found {len(top_paths_by_bs)} different batch sizes in this tournament:"
        )
        for bs in sorted(top_paths_by_bs.keys(), key=lambda x: int(x)):
            print(f"  BS_{bs}: {len(top_paths_by_bs[bs])} checkpoints")
        print("\n")

else:
    print("No Swiss tournament data available")

Found 4 tournament(s)

TOURNAMENT: 20251031_164218
Total checkpoints: 1001

TOP 10 CHECKPOINTS PER BATCH SIZE
--------------------------------------------------------------------------------

Batch Size 512:
  1. Rank   1 | Epoch    0 | Score  69.0 | CHECKPOINTS\E03_BATCH_SIZE512\20251030_1735-E03_BATCH_SIZE512_E_0001.pt
  2. Rank   2 | Epoch    0 | Score  66.0 | CHECKPOINTS\E03_BATCH_SIZE512\20251030_1736-E03_BATCH_SIZE512_E_0013.pt
  3. Rank   3 | Epoch    0 | Score  66.0 | CHECKPOINTS\E03_BATCH_SIZE512\20251030_1735-E03_BATCH_SIZE512_E_0002.pt
  4. Rank   4 | Epoch    0 | Score  63.5 | CHECKPOINTS\E03_BATCH_SIZE512\20251030_1820-E03_BATCH_SIZE512_E_0399.pt
  5. Rank   5 | Epoch    0 | Score  62.0 | CHECKPOINTS\E03_BATCH_SIZE512\20251030_1735-E03_BATCH_SIZE512_E_0000.pt
  6. Rank   6 | Epoch    0 | Score  62.0 | CHECKPOINTS\E03_BATCH_SIZE512\20251030_1829-E03_BATCH_SIZE512_E_0476.pt
  7. Rank   7 | Epoch    0 | Score  62.0 | CHECKPOINTS\E03_BATCH_SIZE512\20251030_1915-E03_BATCH_SIZE5

In [157]:
LR = [
    9.9999755727492e-05,
    9.999902291237878e-05,
    9.999780156189302e-05,
    9.999609168808891e-05,
    9.999389330784221e-05,
    9.999120644285006e-05,
    9.998803111963074e-05,
    9.998436736952336e-05,
    9.998021522868771e-05,
    9.997557473810373e-05,
    9.997044594357117e-05,
    9.996482889570917e-05,
    9.995872364995573e-05,
    9.99521302665672e-05,
    9.994504881061754e-05,
    9.993747935199789e-05,
    9.992942196541575e-05,
    9.992087673039428e-05,
    9.99118437312715e-05,
    9.990232305719945e-05,
    9.989231480214336e-05,
    9.988181906488065e-05,
    9.987083594900002e-05,
    9.985936556290038e-05,
    9.984740801978982e-05,
    9.983496343768446e-05,
    9.98220319394073e-05,
    9.980861365258703e-05,
    9.979470870965668e-05,
    9.978031724785244e-05,
    9.976543940921224e-05,
    9.975007534057434e-05,
    9.973422519357587e-05,
    9.971788912465142e-05,
    9.970106729503138e-05,
    9.968375987074039e-05,
    9.966596702259576e-05,
    9.964768892620573e-05,
    9.962892576196774e-05,
    9.960967771506662e-05,
    9.958994497547283e-05,
    9.956972773794053e-05,
    9.954902620200571e-05,
    9.952784057198417e-05,
    9.950617105696954e-05,
    9.948401787083116e-05,
    9.946138123221205e-05,
    9.943826136452668e-05,
    9.941465849595883e-05,
    9.939057285945926e-05,
    9.936600469274349e-05,
    9.934095423828941e-05,
    9.93154217433349e-05,
    9.928940745987535e-05,
    9.926291164466125e-05,
    9.923593455919561e-05,
    9.920847646973136e-05,
    9.918053764726877e-05,
    9.915211836755271e-05,
    9.912321891107005e-05,
    9.90938395630467e-05,
    9.906398061344499e-05,
    9.90336423569607e-05,
    9.900282509302016e-05,
    9.897152912577736e-05,
    9.893975476411081e-05,
    9.890750232162067e-05,
    9.887477211662551e-05,
    9.884156447215925e-05,
    9.880787971596794e-05,
    9.877371818050649e-05,
    9.873908020293551e-05,
    9.870396612511781e-05,
    9.86683762936152e-05,
    9.863231105968491e-05,
    9.859577077927629e-05,
    9.85587558130271e-05,
    9.852126652626016e-05,
    9.848330328897956e-05,
    9.844486647586715e-05,
    9.840595646627875e-05,
    9.836657364424045e-05,
    9.832671839844478e-05,
    9.828639112224698e-05,
    9.824559221366091e-05,
    9.82043220753554e-05,
    9.816258111465e-05,
    9.812036974351116e-05,
    9.807768837854808e-05,
    9.80345374410086e-05,
    9.799091735677504e-05,
    9.794682855636002e-05,
    9.790227147490221e-05,
    9.785724655216202e-05,
    9.781175423251724e-05,
    9.776579496495873e-05,
    9.771936920308587e-05,
    9.767247740510224e-05,
    9.76251200338109e-05,
    9.757729755661001e-05,
    9.75290104454881e-05,
    9.748025917701948e-05,
    9.743104423235947e-05,
    9.738136609723972e-05,
    9.733122526196337e-05,
    9.728062222140022e-05,
    9.722955747498185e-05,
    9.717803152669668e-05,
    9.712604488508502e-05,
    9.707359806323405e-05,
    9.702069157877271e-05,
    9.696732595386666e-05,
    9.691350171521305e-05,
    9.685921939403541e-05,
    9.680447952607831e-05,
    9.674928265160219e-05,
    9.669362931537786e-05,
    9.663752006668133e-05,
    9.658095545928821e-05,
    9.652393605146834e-05,
    9.646646240598025e-05,
    9.640853509006563e-05,
    9.63501546754437e-05,
    9.629132173830557e-05,
    9.62320368593086e-05,
    9.617230062357059e-05,
    9.611211362066408e-05,
    9.60514764446105e-05,
    9.599038969387426e-05,
    9.592885397135697e-05,
    9.586686988439136e-05,
    9.580443804473534e-05,
    9.574155906856594e-05,
    9.567823357647331e-05,
    9.561446219345446e-05,
    9.555024554890721e-05,
    9.548558427662392e-05,
    9.542047901478521e-05,
    9.535493040595377e-05,
    9.528893909706789e-05,
    9.522250573943516e-05,
    9.515563098872597e-05,
    9.508831550496714e-05,
    9.502055995253532e-05,
    9.495236500015042e-05,
    9.488373132086913e-05,
    9.481465959207813e-05,
    9.47451504954875e-05,
    9.467520471712399e-05,
    9.460482294732415e-05,
    9.453400588072768e-05,
    9.44627542162704e-05,
    9.439106865717749e-05,
    9.431894991095648e-05,
    9.424639868939028e-05,
    9.417341570853013e-05,
    9.410000168868863e-05,
    9.402615735443247e-05,
    9.395188343457548e-05,
    9.387718066217119e-05,
    9.380204977450587e-05,
    9.3726491513091e-05,
    9.365050662365615e-05,
    9.357409585614147e-05,
    9.349725996469042e-05,
    9.34199997076422e-05,
    9.334231584752438e-05,
    9.326420915104529e-05,
    9.318568038908648e-05,
    9.310673033669515e-05,
    9.302735977307642e-05,
    9.294756948158572e-05,
    9.286736024972102e-05,
    9.278673286911505e-05,
    9.27056881355275e-05,
    9.262422684883719e-05,
    9.25423498130341e-05,
    9.246005783621156e-05,
    9.237735173055813e-05,
    9.229423231234969e-05,
    9.221070040194135e-05,
    9.212675682375932e-05,
    9.204240240629284e-05,
    9.195763798208598e-05,
    9.187246438772934e-05,
    9.178688246385193e-05,
    9.170089305511279e-05,
    9.161449701019268e-05,
    9.152769518178565e-05,
    9.144048842659075e-05,
    9.135287760530343e-05,
    9.12648635826071e-05,
    9.117644722716467e-05,
    9.108762941160986e-05,
    9.099841101253862e-05,
    9.090879291050057e-05,
    9.081877598999017e-05,
    9.07283611394381e-05,
    9.063754925120244e-05,
    9.054634122155985e-05,
    9.045473795069676e-05,
    9.036274034270049e-05,
    9.027034930555029e-05,
    9.017756575110836e-05,
    9.008439059511096e-05,
    8.999082475715923e-05,
    8.989686916071025e-05,
    8.980252473306781e-05,
    8.970779240537335e-05,
    8.961267311259665e-05,
    8.951716779352674e-05,
    8.94212773907626e-05,
    8.932500285070377e-05,
    8.922834512354107e-05,
    8.913130516324728e-05,
    8.903388392756762e-05,
    8.893608237801034e-05,
    8.883790147983729e-05,
    8.873934220205424e-05,
    8.864040551740151e-05,
    8.854109240234424e-05,
    8.844140383706278e-05,
    8.834134080544299e-05,
    8.824090429506663e-05,
    8.814009529720149e-05,
    8.803891480679169e-05,
    8.793736382244782e-05,
    8.78354433464371e-05,
    8.773315438467347e-05,
    8.763049794670771e-05,
    8.752747504571738e-05,
    8.742408669849697e-05,
    8.732033392544769e-05,
    8.721621775056754e-05,
    8.711173920144112e-05,
    8.700689930922956e-05,
    8.690169910866023e-05,
    8.679613963801666e-05,
    8.66902219391282e-05,
    8.65839470573598e-05,
    8.647731604160163e-05,
    8.637032994425875e-05,
    8.626298982124075e-05,
    8.615529673195132e-05,
    8.60472517392778e-05,
    8.593885590958059e-05,
    8.58301103126828e-05,
    8.572101602185957e-05,
    8.561157411382751e-05,
    8.550178566873404e-05,
    8.539165177014683e-05,
    8.528117350504297e-05,
    8.517035196379834e-05,
    8.505918824017684e-05,
    8.494768343131948e-05,
    8.483583863773377e-05,
    8.472365496328265e-05,
    8.461113351517368e-05,
    8.449827540394814e-05,
    8.438508174347002e-05,
    8.427155365091506e-05,
    8.415769224675969e-05,
    8.404349865477e-05,
    8.392897400199066e-05,
    8.381411941873374e-05,
    8.36989360385676e-05,
    8.358342499830572e-05,
    8.346758743799542e-05,
    8.335142450090666e-05,
    8.323493733352072e-05,
    8.311812708551895e-05,
    8.300099490977129e-05,
    8.288354196232507e-05,
    8.276576940239345e-05,
    8.264767839234405e-05,
    8.252927009768746e-05,
    8.241054568706573e-05,
    8.22915063322409e-05,
    8.21721532080833e-05,
    8.205248749256012e-05,
    8.193251036672364e-05,
    8.181222301469965e-05,
    8.169162662367575e-05,
    8.157072238388965e-05,
    8.144951148861738e-05,
    8.132799513416155e-05,
    8.12061745198395e-05,
    8.108405084797153e-05,
    8.096162532386898e-05,
    8.083889915582234e-05,
    8.071587355508931e-05,
    8.059254973588296e-05,
    8.046892891535955e-05,
    8.034501231360671e-05,
    8.022080115363123e-05,
    8.009629666134715e-05,
    7.997150006556353e-05,
    7.98464125979724e-05,
    7.972103549313654e-05,
    7.959536998847737e-05,
    7.94694173242627e-05,
    7.934317874359447e-05,
    7.921665549239651e-05,
    7.90898488194022e-05,
    7.896275997614226e-05,
    7.883539021693221e-05,
    7.870774079886019e-05,
    7.857981298177441e-05,
    7.845160802827077e-05,
    7.832312720368041e-05,
    7.819437177605721e-05,
    7.806534301616525e-05,
    7.79360421974663e-05,
    7.780647059610726e-05,
    7.767662949090747e-05,
    7.754652016334627e-05,
    7.741614389755017e-05,
    7.728550198028029e-05,
    7.715459570091961e-05,
    7.702342635146026e-05,
    7.689199522649077e-05,
    7.676030362318327e-05,
    7.662835284128071e-05,
    7.649614418308406e-05,
    7.636367895343938e-05,
    7.623095845972503e-05,
    7.60979840118387e-05,
    7.596475692218451e-05,
    7.583127850566001e-05,
    7.569755007964331e-05,
    7.556357296398e-05,
    7.542934848097009e-05,
    7.529487795535508e-05,
    7.516016271430472e-05,
    7.50252040874041e-05,
    7.489000340664036e-05,
    7.475456200638965e-05,
    7.461888122340388e-05,
    7.448296239679764e-05,
    7.434680686803484e-05,
    7.42104159809156e-05,
    7.407379108156292e-05,
    7.393693351840939e-05,
    7.37998446421839e-05,
    7.366252580589832e-05,
    7.352497836483412e-05,
    7.338720367652902e-05,
    7.324920310076359e-05,
    7.311097799954776e-05,
    7.29725297371075e-05,
    7.283385967987126e-05,
    7.269496919645654e-05,
    7.255585965765628e-05,
    7.241653243642555e-05,
    7.227698890786773e-05,
    7.21372304492211e-05,
    7.199725843984526e-05,
    7.18570742612074e-05,
    7.171667929686878e-05,
    7.157607493247103e-05,
    7.143526255572245e-05,
    7.129424355638433e-05,
    7.11530193262573e-05,
    7.10115912591675e-05,
    7.08699607509528e-05,
    7.07281291994492e-05,
    7.058609800447681e-05,
    7.044386856782623e-05,
    7.030144229324453e-05,
    7.015882058642157e-05,
    7.001600485497599e-05,
    6.987299650844145e-05,
    6.972979695825256e-05,
    6.958640761773107e-05,
    6.944282990207189e-05,
    6.92990652283291e-05,
    6.9155115015402e-05,
    6.901098068402106e-05,
    6.886666365673398e-05,
    6.872216535789151e-05,
    6.857748721363355e-05,
    6.843263065187496e-05,
    6.828759710229154e-05,
    6.814238799630586e-05,
    6.799700476707319e-05,
    6.785144884946728e-05,
    6.770572168006628e-05,
    6.755982469713855e-05,
    6.741375934062837e-05,
    6.726752705214188e-05,
    6.712112927493269e-05,
    6.697456745388779e-05,
    6.682784303551317e-05,
    6.668095746791962e-05,
    6.653391220080835e-05,
    6.63867086854568e-05,
    6.62393483747043e-05,
    6.609183272293754e-05,
    6.594416318607649e-05,
    6.579634122155984e-05,
    6.564836828833073e-05,
    6.550024584682225e-05,
    6.535197535894311e-05,
    6.520355828806315e-05,
    6.505499609899897e-05,
    6.490629025799938e-05,
    6.475744223273099e-05,
    6.460845349226374e-05,
    6.445932550705633e-05,
    6.431005974894178e-05,
    6.416065769111284e-05,
    6.401112080810753e-05,
    6.386145057579449e-05,
    6.371164847135852e-05,
    6.356171597328589e-05,
    6.341165456134982e-05,
    6.326146571659589e-05,
    6.311115092132734e-05,
    6.296071165909052e-05,
    6.281014941466023e-05,
    6.265946567402504e-05,
    6.250866192437259e-05,
    6.235773965407506e-05,
    6.22067003526743e-05,
    6.205554551086725e-05,
    6.190427662049116e-05,
    6.175289517450893e-05,
    6.160140266699431e-05,
    6.144980059311716e-05,
    6.12980904491288e-05,
    6.114627373234705e-05,
    6.0994351941141654e-05,
    6.084232657491931e-05,
    6.069019913410907e-05,
    6.05379711201473e-05,
    6.038564403546309e-05,
    6.023321938346322e-05,
    6.008069866851752e-05,
    5.9928083395943843e-05,
    5.97753750719933e-05,
    5.962257520383544e-05,
    5.946968529954323e-05,
    5.9316706868078325e-05,
    5.9163641419276094e-05,
    5.90104904638307e-05,
    5.885725551328025e-05,
    5.8703938079991856e-05,
    5.855053967714667e-05,
    5.839706181872499e-05,
    5.8243506019491355e-05,
    5.808987379497947e-05,
    5.79361666614774e-05,
    5.778238613601247e-05,
    5.762853373633641e-05,
    5.7474610980910266e-05,
    5.7320619388889516e-05,
    5.716656048010898e-05,
    5.7012435775067916e-05,
    5.6858246794914925e-05,
    5.6703995061432986e-05,
    5.654968209702445e-05,
    5.639530942469598e-05,
    5.624087856804352e-05,
    5.608639105123726e-05,
    5.5931848399006664e-05,
    5.5777252136625284e-05,
    5.562260378989584e-05,
    5.546790488513505e-05,
    5.53131569491587e-05,
    5.515836150926637e-05,
    5.5003520093226626e-05,
    5.4848634229261714e-05,
    5.469370544603254e-05,
    5.453873527262372e-05,
    5.4383725238528246e-05,
    5.422867687363259e-05,
    5.407359170820152e-05,
    5.3918471272863014e-05,
    5.376331709859311e-05,
    5.3608130716700924e-05,
    5.345291365881336e-05,
    5.3297667456860115e-05,
    5.3142393643058546e-05,
    5.2987093749898496e-05,
    5.283176931012721e-05,
    5.267642185673421e-05,
    5.2521052922936155e-05,
    5.236566404216166e-05,
    5.221025674803629e-05,
    5.205483257436725e-05,
    5.189939305512842e-05,
    5.1743939724445106e-05,
    5.1588474116578904e-05,
    5.143299776591261e-05,
    5.1277512206935026e-05,
    5.112201897422586e-05,
    5.096651960244053e-05,
    5.0811015626295077e-05,
    5.065550858055094e-05,
    5.04999999999999e-05,
    5.034449141944886e-05,
    5.018898437370473e-05,
    5.003348039755927e-05,
    4.987798102577394e-05,
    4.972248779306478e-05,
    4.9567002234087184e-05,
    4.9411525883420894e-05,
    4.925606027555469e-05,
    4.910060694487138e-05,
    4.894516742563255e-05,
    4.878974325196353e-05,
    4.863433595783814e-05,
    4.847894707706366e-05,
    4.83235781432656e-05,
    4.8168230689872583e-05,
    4.8012906250101316e-05,
    4.785760635694127e-05,
    4.77023325431397e-05,
    4.754708634118646e-05,
    4.739186928329891e-05,
    4.72366829014067e-05,
    4.7081528727136826e-05,
    4.692640829179831e-05,
    4.677132312636723e-05,
    4.661627476147159e-05,
    4.6461264727376116e-05,
    4.630629455396728e-05,
    4.6151365770738126e-05,
    4.599647990677321e-05,
    4.584163849073345e-05,
    4.5686843050841156e-05,
    4.553209511486478e-05,
    4.5377396210104e-05,
    4.522274786337454e-05,
    4.506815160099317e-05,
    4.4913608948762565e-05,
    4.475912143195632e-05,
    4.460469057530386e-05,
    4.445031790297538e-05,
    4.429600493856685e-05,
    4.414175320508492e-05,
    4.3987564224931924e-05,
    4.383343951989086e-05,
    4.367938061111033e-05,
    4.352538901908957e-05,
    4.337146626366343e-05,
    4.321761386398737e-05,
    4.306383333852243e-05,
    4.2910126205020366e-05,
    4.2756493980508484e-05,
    4.260293818127484e-05,
    4.244946032285317e-05,
    4.229606192000799e-05,
    4.2142744486719584e-05,
    4.198950953616913e-05,
    4.183635858072375e-05,
    4.168329313192151e-05,
    4.153031470045662e-05,
    4.1377424796164425e-05,
    4.122462492800656e-05,
    4.107191660405603e-05,
    4.0919301331482355e-05,
    4.076678061653663e-05,
    4.0614355964536785e-05,
    4.046202887985256e-05,
    4.030980086589079e-05,
    4.015767342508055e-05,
    4.000564805885822e-05,
    3.9853726267652815e-05,
    3.9701909550871074e-05,
    3.955019940688271e-05,
    3.939859733300556e-05,
    3.9247104825490944e-05,
    3.909572337950871e-05,
    3.894445448913263e-05,
    3.879329964732557e-05,
    3.864226034592481e-05,
    3.8491338075627275e-05,
    3.834053432597484e-05,
    3.818985058533964e-05,
    3.803928834090933e-05,
    3.788884907867253e-05,
    3.773853428340398e-05,
    3.758834543865005e-05,
    3.743828402671399e-05,
    3.7288351528641366e-05,
    3.713854942420538e-05,
    3.698887919189234e-05,
    3.6839342308887044e-05,
    3.668994025105809e-05,
    3.6540674492943546e-05,
    3.639154650773613e-05,
    3.624255776726888e-05,
    3.609370974200049e-05,
    3.594500390100091e-05,
    3.5796441711936715e-05,
    3.564802464105678e-05,
    3.5499754153177644e-05,
    3.5351631711669154e-05,
    3.520365877844006e-05,
    3.505583681392342e-05,
    3.4908167277062365e-05,
    3.476065162529561e-05,
    3.461329131454309e-05,
    3.4466087799191554e-05,
    3.43190425320803e-05,
    3.4172156964486734e-05,
    3.4025432546112115e-05,
    3.3878870725067206e-05,
    3.373247294785802e-05,
    3.358624065937152e-05,
    3.344017530286135e-05,
    3.329427831993362e-05,
    3.314855115053262e-05,
    3.300299523292672e-05,
    3.2857612003694034e-05,
    3.271240289770836e-05,
    3.2567369348124935e-05,
    3.242251278636635e-05,
    3.227783464210839e-05,
    3.2133336343265914e-05,
    3.1989019315978824e-05,
    3.184488498459788e-05,
    3.1700934771670786e-05,
    3.1557170097927994e-05,
    3.1413592382268813e-05,
    3.127020304174732e-05,
    3.1127003491558424e-05,
    3.098399514502385e-05,
    3.084117941357828e-05,
    3.0698557706755326e-05,
    3.055613143217362e-05,
    3.041390199552303e-05,
    3.027187080055065e-05,
    3.0130039249047037e-05,
    2.998840874083236e-05,
    2.9846980673742552e-05,
    2.9705756443615514e-05,
    2.956473744427742e-05,
    2.9423925067528827e-05,
    2.9283320703131075e-05,
    2.9142925738792464e-05,
    2.9002741560154596e-05,
    2.8862769550778755e-05,
    2.872301109213212e-05,
    2.8583467563574305e-05,
    2.8444140342343555e-05,
    2.8305030803543334e-05,
    2.8166140320128588e-05,
    2.802747026289236e-05,
    2.7889022000452113e-05,
    2.7750796899236277e-05,
    2.7612796323470817e-05,
    2.7475021635165736e-05,
    2.733747419410155e-05,
    2.7200155357815978e-05,
    2.7063066481590474e-05,
    2.6926208918436932e-05,
    2.6789584019084257e-05,
    2.665319313196503e-05,
    2.6517037603202226e-05,
    2.6381118776595988e-05,
    2.6245437993610236e-05,
    2.6109996593359486e-05,
    2.5974795912595754e-05,
    2.5839837285695137e-05,
    2.5705122044644805e-05,
    2.5570651519029774e-05,
    2.5436427036019886e-05,
    2.5302449920356555e-05,
    2.5168721494339866e-05,
    2.503524307781536e-05,
    2.4902015988161156e-05,
    2.4769041540274827e-05,
    2.463632104656047e-05,
    2.450385581691579e-05,
    2.4371647158719152e-05,
    2.4239696376816613e-05,
    2.4108004773509103e-05,
    2.3976573648539615e-05,
    2.3845404299080274e-05,
    2.3714498019719587e-05,
    2.3583856102449705e-05,
    2.3453479836653627e-05,
    2.3323370509092428e-05,
    2.3193529403892648e-05,
    2.3063957802533594e-05,
    2.2934656983834635e-05,
    2.280562822394269e-05,
    2.267687279631948e-05,
    2.2548391971729138e-05,
    2.242018701822551e-05,
    2.229225920113974e-05,
    2.2164609783067695e-05,
    2.203724002385766e-05,
    2.1910151180597712e-05,
    2.1783344507603402e-05,
    2.1656821256405438e-05,
    2.153058267573721e-05,
    2.1404630011522532e-05,
    2.1278964506863363e-05,
    2.1153587402027517e-05,
    2.102849993443639e-05,
    2.0903703338652784e-05,
    2.0779198846368698e-05,
    2.0654987686393222e-05,
    2.0531071084640385e-05,
    2.040745026411698e-05,
    2.0284126444910627e-05,
    2.0161100844177622e-05,
    2.003837467613098e-05,
    1.9915949152028388e-05,
    1.9793825480160425e-05,
    1.9672004865838388e-05,
    1.955048851138254e-05,
    1.9429277616110283e-05,
    1.9308373376324188e-05,
    1.918777698530029e-05,
    1.906748963327629e-05,
    1.8947512507439804e-05,
    1.8827846791916624e-05,
    1.8708493667759034e-05,
    1.858945431293418e-05,
    1.8470729902312447e-05,
    1.835232160765586e-05,
    1.823423059760644e-05,
    1.8116458037674826e-05,
    1.799900509022861e-05,
    1.7881872914480977e-05,
    1.7765062666479174e-05,
    1.764857549909325e-05,
    1.7532412562004498e-05,
    1.7416575001694205e-05,
    1.730106396143231e-05,
    1.718588058126618e-05,
    1.7071025998009243e-05,
    1.695650134522988e-05,
    1.68423077532402e-05,
    1.6728446349084836e-05,
    1.6614918256529876e-05,
    1.650172459605175e-05,
    1.6388866484826198e-05,
    1.6276345036717237e-05,
    1.616416136226612e-05,
    1.6052316568680393e-05,
    1.594081175982306e-05,
    1.582964803620155e-05,
    1.5718826494956903e-05,
    1.5608348229853055e-05,
    1.5498214331265846e-05,
    1.5388425886172397e-05,
    1.527898397814032e-05,
    1.5169889687317096e-05,
    1.50611440904193e-05,
    1.4952748260722104e-05,
    1.4844703268048552e-05,
    1.4737010178759131e-05,
    1.4629670055741147e-05,
    1.4522683958398262e-05,
    1.4416052942640073e-05,
    1.4309778060871678e-05,
    1.420386036198324e-05,
    1.4098300891339664e-05,
    1.3993100690770347e-05,
    1.3888260798558784e-05,
    1.3783782249432362e-05,
    1.367966607455221e-05,
    1.3575913301502938e-05,
    1.347252495428253e-05,
    1.336950205329221e-05,
    1.3266845615326447e-05,
    1.3164556653562815e-05,
    1.3062636177552096e-05,
    1.2961085193208218e-05,
    1.2859904702798425e-05,
    1.2759095704933297e-05,
    1.2658659194556936e-05,
    1.2558596162937134e-05,
    1.2458907597655666e-05,
    1.2359594482598395e-05,
    1.2260657797945669e-05,
    1.2162098520162641e-05,
    1.2063917621989581e-05,
    1.1966116072432304e-05,
    1.1868694836752634e-05,
    1.1771654876458848e-05,
    1.167499714929617e-05,
    1.1578722609237334e-05,
    1.1482832206473182e-05,
    1.1387326887403284e-05,
    1.1292207594626593e-05,
    1.1197475266932109e-05,
    1.1103130839289676e-05,
    1.1009175242840702e-05,
    1.0915609404888993e-05,
    1.0822434248891563e-05,
    1.0729650694449645e-05,
    1.0637259657299429e-05,
    1.0545262049303159e-05,
    1.045365877844007e-05,
    1.0362450748797486e-05,
    1.0271638860561814e-05,
    1.0181224010009745e-05,
    1.009120708949936e-05,
    1.0001588987461313e-05,
    9.912370588390085e-06,
    9.823552772835255e-06,
    9.735136417392808e-06,
    9.647122394696489e-06,
    9.55951157340917e-06,
    9.472304818214256e-06,
    9.385502989807248e-06,
    9.299106944887144e-06,
    9.213117536147988e-06,
    9.127535612270593e-06,
    9.04236201791396e-06,
    8.957597593707086e-06,
    8.873243176240606e-06,
    8.789299598058588e-06,
    8.705767687650229e-06,
    8.622648269441791e-06,
    8.539942163788355e-06,
    8.45765018696581e-06,
    8.375773151162735e-06,
    8.294311864472413e-06,
    8.213267130884862e-06,
    8.132639750278895e-06,
    8.0524305184142e-06,
    7.972640226923498e-06,
    7.893269663304774e-06,
    7.814319610913443e-06,
    7.735790848954622e-06,
    7.657684152475538e-06,
    7.580000292357723e-06,
    7.502740035309522e-06,
    7.425904143858459e-06,
    7.3494933763437965e-06,
    7.273508486908934e-06,
    7.197950225494073e-06,
    7.122819337828735e-06,
    7.0481165654244645e-06,
    6.973842645567456e-06,
    6.899998311311305e-06,
    6.826584291469791e-06,
    6.753601310609657e-06,
    6.6810500890434545e-06,
    6.608931342822436e-06,
    6.537245783729533e-06,
    6.465994119272264e-06,
    6.395177052675779e-06,
    6.3247952828759435e-06,
    6.254849504512428e-06,
    6.185340407921812e-06,
    6.116268679130809e-06,
    6.0476349998495155e-06,
    5.979440047464617e-06,
    5.911684495032782e-06,
    5.8443690112739455e-06,
    5.777494260564763e-06,
    5.711060902932029e-06,
    5.645069594046149e-06,
    5.5795209852146965e-06,
    5.514415723376001e-06,
    5.449754451092703e-06,
    5.385537806545446e-06,
    5.3217664235266015e-06,
    5.2584409314339645e-06,
    5.195561955264571e-06,
    5.1331301156085365e-06,
    5.071146028642922e-06,
    5.009610306125638e-06,
    4.948523555389416e-06,
    4.88788637933582e-06,
    4.827699376429302e-06,
    4.767963140691296e-06,
    4.708678261694317e-06,
    4.649845324556194e-06,
    4.591464909934258e-06,
    4.533537594019644e-06,
    4.476063948531545e-06,
    4.419044540711673e-06,
    4.362479933318551e-06,
    4.306370684622022e-06,
    4.2507173483976985e-06,
    4.195520473921564e-06,
    4.140780605964476e-06,
    4.086498284786824e-06,
    4.032674046133228e-06,
    3.979308421227172e-06,
    3.92640193676584e-06,
    3.8739551149148635e-06,
    3.821968473303201e-06,
    3.770442525018037e-06,
    3.719377778599663e-06,
    3.6687747380365037e-06,
    3.618633902760157e-06,
    3.5689557676404173e-06,
    3.519740822980409e-06,
    3.470989554511793e-06,
    3.4227024433898977e-06,
    3.3748799661890104e-06,
    3.3275225948976706e-06,
    3.280630796914023e-06,
    3.2342050350411726e-06,
    3.188245767482658e-06,
    3.1427534478378875e-06,
    3.0977285250976994e-06,
    3.053171443639896e-06,
    3.0090826432248825e-06,
    2.965462558991318e-06,
    2.9223116214518316e-06,
    2.8796302564887524e-06,
    2.8374188853499126e-06,
    2.79567792464452e-06,
    2.754407786339001e-06,
    2.7136088777529433e-06,
    2.6732816015551187e-06,
    2.633426355759468e-06,
    2.594043533721166e-06,
    2.555133524132762e-06,
    2.5166967110203525e-06,
    2.4787334737397637e-06,
    2.4412441869728165e-06,
    2.4042292207236388e-06,
    2.367688940315011e-06,
    2.3316237063847377e-06,
    2.2960338748821268e-06,
    2.260919797064427e-06,
    2.2262818194934438e-06,
    2.1921202840320077e-06,
    2.1584355278406838e-06,
    2.1252278833744233e-06,
    2.0924976783792752e-06,
    2.060245235889133e-06,
    2.0284708742225903e-06,
    1.9971749069797807e-06,
    1.966357643039257e-06,
    1.9360193865549688e-06,
    1.9061604369532599e-06,
    1.876781088929909e-06,
    1.8478816324472317e-06,
    1.8194623527311793e-06,
    1.7915235302685858e-06,
    1.7640654408043367e-06,
    1.7370883553386942e-06,
    1.7105925401245973e-06,
    1.684578256665052e-06,
    1.6590457617105327e-06,
    1.6339953072564527e-06,
    1.6094271405406865e-06,
    1.585341504041119e-06,
    1.56173863547326e-06,
    1.5386187677878922e-06,
    1.5159821291687794e-06,
    1.4938289430304026e-06,
    1.472159428015767e-06,
    1.4509737979942366e-06,
    1.4302722620594245e-06,
    1.4100550245271424e-06,
    1.3903222849333511e-06,
    1.371074238032226e-06,
    1.3523110737942278e-06,
    1.3340329774042015e-06,
    1.3162401292595847e-06,
    1.2989327049686046e-06,
    1.2821108753485534e-06,
    1.2657748064240998e-06,
    1.2499246594256415e-06,
    1.2345605907877383e-06,
    1.2196827521475403e-06,
    1.2052912903433102e-06,
    1.191386347412961e-06,
    1.1779680605926767e-06,
    1.1650365623155216e-06,
    1.152591980210166e-06,
    1.1406344370996105e-06,
    1.1291640509999774e-06,
    1.1181809351193513e-06,
    1.1076851978566465e-06,
    1.0976769428005578e-06,
    1.0881562687285157e-06,
    1.0791232696057363e-06,
    1.0705780345842656e-06,
    1.0625206480021353e-06,
    1.0549511893824897e-06,
    1.047869733432827e-06,
    1.0412763500442656e-06,
    1.0351711042908343e-06,
    1.0295540564288416e-06,
    1.0244252618962857e-06,
    1.0197847713123014e-06,
    1.0156326304766528e-06,
    1.0119688803692896e-06,
    1.008793557149956e-06,
    1.0061066921578007e-06,
    1.0039083119111023e-06,
    1.0021984381069834e-06,
    1.0009770876212127e-06,
    1.0002442725080184e-06,
    1e-06,
]

In [ ]:
# Visualize the learning rate schedule
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=list(range(len(LR))),
        y=LR,
        mode="lines",
        name="Learning Rate",
        line=dict(color="blue", width=2),
    )
)

fig.update_layout(
    title="Learning Rate Schedule over Epochs",
    xaxis_title="Epoch",
    yaxis_title="Learning Rate",
    height=500,
    hovermode="x unified",
)

fig.show()

# Also show log scale version for better visibility
fig_log = go.Figure()

fig_log.add_trace(
    go.Scatter(
        x=list(range(len(LR))),
        y=LR,
        mode="lines",
        name="Learning Rate",
        line=dict(color="blue", width=2),
    )
)

fig_log.update_layout(
    title="Learning Rate Schedule over Epochs (Log Scale)",
    xaxis_title="Epoch",
    yaxis_title="Learning Rate (log scale)",
    yaxis_type="log",
    height=500,
    hovermode="x unified",
)

fig_log.show()